# Week 1: Audio

## Week plan
1. Physics of sound
    - Soundwaves (how to describe them?)
    - Record audio (how?)
2. Digital representation of audio
3. Analyzing audio signals
    - Fourier transform
    - Spectrogram (what pitches are present in the audio)
4. Fingerprinting the spectrograms (only record a few important points of the audio file to quickly identify files)

## Physics of Sound
Sound is a domino effect of air molecules colliding with each other. It's a constant oscillation of air pressure. If we freeze time or space, pressure would look a bit like this: 
![Px](images/Px.png)
![PT](images/PT.png)

Quick cheat sheet:
* $\lambda$: wavelength
* $T$: period (how long for one oscillation)
* $f$: frequency
* $V$: velocity
* $A$: amplitude (in units of pressure)

### Important formulas
1. $V = \frac{\lambda}{T} = \lambda f$
2. $f = \frac{1}{T}$
3. $P(x,t) = A \sin(2 \pi \frac{f}{v} x  - 2 \pi ft)$ (general formula to describe waves - notice how it behaves when you fix one of $x$ or $t$)
       
With a microphone that is (more or less) fixed in space, we can fix $x$ as $x_0$ (wherever $x_0$ is) and simply simplify it, by letting $\phi = 2\pi \frac{f}{v} x_0$ and flipping the other term, into

4. $P(t) = A \sin(\phi + 2\pi ft)$

### Microphone
![Micrphone](images/microphone.png)


## Digital Representation of Sound
We can't store infinitely many points to represent the function $f(x)=\sin(2\pi x)$

Sampling rate = how often (in terms of the X-axis) do you pick a point on the graph to save (typically 44.1 kHz, since humans can hear up to 20 kHz - this ensures that you are almost usually aligned)

Bit depth = how many bits do you spend to represent a voltage (because, again, voltage is a real number and can get super precise) (typically 16 bits on CD)

What you often do is that you look at the whole recording and normalize it into a range

## Fourier Analysis
For any periodic f, we can write:

$$
f(x) =  
    \sum_{k=-\infty}^{\infty} a_k \sin(2\pi \frac{k}{L} x) + b_k \cos(2\pi \frac{k}{L} x)
$$


Periodic as in $f(x) = f(x+L)$ for some period $L$. 

Even if $f$ were not periodic, there is a method to still represent it (involving integrals)

### Continuous Fourier Transform
Using euler's formula $e^{ix} = \cos(x) + i\sin(x)$, we can simplify the series into:  
$$
f(x) = 
    \sum_{k=-\infty}^{\infty} \gamma_k e^{i2\pi\frac{k}{L} x}
$$
    
$\gamma$ itself is also complex valued. 

### Discrete Fourier Transform (Type I)
We write Type I because there are other types, but this is the only kind we'd concern ourselves with.  
We care about discrete fourier transform because our sampling is discrete.    

Essentially, what we're doing is to fit N sample points with N component waves. If the whole sample spans length L, each component wave oscillates an integer number of times throughout length L (in other words, it has period L/m, where m is an integer). The first component wave is just a line, the second has period L, the third period L/2, the fourth period L/3, etc., so the last one has period L/(N-1). 
  

To take N samples, we take 
$x_n = \frac{n}{N} L$   $(n=0,1,...,N-1)$  , which means that we take the left values. It's important that you sample at **regular intervals**.  

Then,  
$$
y_n = \frac{1}{N} \sum _{k=0}^{N-1} c_k e^{i2\pi \frac{k}{L} x_n} 
$$

(We limit $k$ to $N-1$ because we can't get any more accurate than that). Note that in this case, L is just the length of the entire sample.  
This takes ${c_k}$ into ${y_k}$ (also known as Inverse Fourier Transform (Type I)).  

Now, we need to figure out the $c$'s ($|c_k|$ is the amplitude of how much each component wave contributes to the final wave).  

This,  
$$
c_k = \sum_{n=0}^{N-1} y_n e^{-i 2\pi \frac{k}{L} x_n} 
$$
takes up from ${y_n}$ into ${c_k}$ (also known as Discrete Fourier Transform (Type I)).  
Note that ${c_k}$ are complex numbers. When you plug them back to compute ${y_k}$ you result in real numbers.  

Also, note that, since we're dealing with real valued samples, $C_{N-r}=\overline{C_r}$, so numpy's Fourier transform only gives $N//2 + 1$ coefficients (to avoid redundant computations). See that this is true by noting that $\frac{k}{L} x_n = \frac{n}{N}k$. 

### Fast Fourier Transform
Basic DFT is of $O(N^2)$, which is catastrophic as it'd take longer to compute the result for a song than it takes to listen to it (under typical encoding schemes). 

The intuition is to keep cutting the range in half more and more (each time you cut it in half, it turns from $N^2$ into $\frac{N^2}{4}$ - rinse and repeat and you get a $log(N)$). The details of the resummation are non-trivial, but that's the idea behind it. 

## Spectrogram
A spectrogram is essentially a "heat map" of the most dominant frequencies mapped over time. Therefore it's plotted with frequency against time. You obtain this map by dividing the entire sample into little chunks of time. Then you per Its resolution is only as good as $N$ - the sampling rate (on one axis) and $\Delta t$ - how fine you cut your time periods. 

So if we always use the same $N$ and $\Delta t$, we don't ever have to convert bin indices back to physical frequencies and $\Delta t$. 

We can find local maximums (refer to `2_PeakFinding` Jupyter notebook). Therefore we have a list of frequencies and times of these local maximums. That is what we're going to use to work out our fingerprints of the audio sample. 

## Matching Spectrogram Fingerprints
When we try to match an audio input with fingerprints of audio files, we cannot guarantee an **absolute** match of the time indices (the audio input could start halfway through the song). Therefore we try to compare the difference in time, as the relative relationship is the same no matter when we start. 

Therefore the encoding scheme would be, given a list of points $\{{f_i,t_i}\}$, to store
$$
((f_1, f_2, t_2-t_1), t_1) \\
((f_1, f_3, t_3-t_1), t_1) \\
... \\
n_{fanout}
$$
where $n_{fanout}$ is some limit we set. 

For each single local max, we fanout and find its relationship with $n_{fanout}$ other maximums and store them as fingerprints. You order them by the distance with this local max and go out $n_{fanout}$ fingerprints down the list. 

We record the extra absolute $t_i$ (that is, relative to the beginning of the song) in the tuple so that when we match the relative relationship, we can calculate the offset of the audio input with the file compared to. Note that the offset should be the same between an audio sample and its match in the database. 

### Database
For the purpose of this course, we store the song fingerprints as a dictionary. Usually you could store into an SQL database with the hash value of the key stored along with the content mapped to.  
The mapping is:  
$$(f_n, f_m, t_m - t_n) \rightarrow [ (song\_id, t_{match}),... ]$$
where each relative relationship maps to a list of songs where this relationship occurs.

### Checking Against Database
Given an audio sample we wish to match, we compute its fingerprints. By feeding each fingerprint into the database, we tally up the number of matches that occurs for each song ID and offset (so the tally should have a two-dimensional key - the song ID and the offset). 

## Week Capstone Project Outline
Audio inputs include
1. mp3 files on our computer
2. microphone
We need to convert both of these inputs into digital audio signals, then into spectrograms, and then finally into peaks. Note that when finding the peaks, we need to use a consistent cutoff frequency so that the fingerprint generated would be the consistent. Then the fingerprints can be either stored into the database or matched against the database. 

Beyond the barebone, we can also test the program automatically. For example, we can cut a full song into short clips and test the success rate. Also, there are lots of tinkerings we can do with the individual steps, such as the peak-finding algorithm. 